In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

ModuleNotFoundError: No module named 'bs4'

In [ ]:
def habr_search(queries):
    search_url = "https://habr.com/ru/search/"

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    results = []
    used_links = set()  

    for query in queries:
         params = {
            "q": query,
            "target_type": "posts"  
        }

        resp = requests.get(search_url, params=params, headers=headers)
        resp.raise_for_status()

        soup = BeautifulSoup(resp.text, "lxml")

        # ищем все карточки материалов в article
        articles = soup.find_all("article")

        for art in articles:
            h = art.find(["h2", "h3"])
            if h is None:
                continue

            title = h.get_text(strip=True)
            a = h.find("a")
            if a is None or a.get("href") is None:
                continue

            link = urljoin("https://habr.com", a["href"])
            t = art.find("time")
            date = t.get("datetime") if t else None
            if link in used_links:
                continue
            used_links.add(link)


            results.append({
                "date": date,
                "title": title,
                "link": link
            })
    df = pd.DataFrame(results)
    return df